In [1]:
import numpy as np
# from time import time
# import matplotlib.pyplot as plt 
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    ol = Overlay("/home/xilinx/jupyter_notebooks/R2plus1D/R2plus1D.bit")
    ip_r2plus1d = ol.r2plus1d_0
    
    print("IP loaded!")

IP loaded!


In [2]:
    # calculate the number of input data
    INPUT = open("/home/xilinx/jupyter_notebooks/R2plus1D/input.dat", "r+")
    inputNum = 0
    line = INPUT.readline()
    while line:
        inputNum = inputNum + 1
        line = INPUT.readline()
    
    print("inputNum:", inputNum)

    # calculate the number of kernel1 weight
    KERNEL1 = open("/home/xilinx/jupyter_notebooks/R2plus1D/Conv3d1weight.dat", "r+")
    kernel1Num = 0
    line = KERNEL1.readline()
    while line:
        kernel1Num = kernel1Num + 1
        line = KERNEL1.readline()
    
    print("kernel1Num:", kernel1Num)
    
    # calculate the number of kernel2 weight
    KERNEL2 = open("/home/xilinx/jupyter_notebooks/R2plus1D/Conv3d2weight.dat", "r+")
    kernel2Num = 0
    line = KERNEL2.readline()
    while line:
        kernel2Num = kernel2Num + 1
        line = KERNEL2.readline()
    
    print("kernel2Num:", kernel2Num)
    
    # allocate memory 
    inputBuffer = allocate(shape=(inputNum,), dtype=float)
    outputBuffer = allocate(shape=(200704,), dtype=float)
    kernel1Buffer = allocate(shape=(kernel1Num,), dtype=float)
    kernel2Buffer = allocate(shape=(kernel2Num,), dtype=float)

    # load input data
    print("loading input data...")
    INPUT.seek(0)
    for i in range(inputNum):
        line = INPUT.readline()
        inputBuffer[i] = line
        if (i % 1000 == 0):
            print(i, "/", inputNum)
    INPUT.close()
    
    # load kernel1 weight
    print("loading kernel1 data...")
    KERNEL1.seek(0)
    for i in range(kernel1Num):
        line = KERNEL1.readline()
        kernel1Buffer[i] = line
        if (i % 1000 == 0):
            print(i, "/", kernel1Num)
    KERNEL1.close()
    
    # load kernel2 weight
    print("loading kernel2 data...")
    KERNEL2.seek(0)
    for i in range(kernel2Num):
        line = KERNEL2.readline()
        kernel2Buffer[i] = line
        if (i % 1000 == 0):
            print(i, "/", kernel2Num)
    KERNEL2.close()

inputNum: 36963
kernel1Num: 6615
kernel2Num: 8640
loading input data...
0 / 36963
1000 / 36963
2000 / 36963
3000 / 36963
4000 / 36963
5000 / 36963
6000 / 36963
7000 / 36963
8000 / 36963
9000 / 36963
10000 / 36963
11000 / 36963
12000 / 36963
13000 / 36963
14000 / 36963
15000 / 36963
16000 / 36963
17000 / 36963
18000 / 36963
19000 / 36963
20000 / 36963
21000 / 36963
22000 / 36963
23000 / 36963
24000 / 36963
25000 / 36963
26000 / 36963
27000 / 36963
28000 / 36963
29000 / 36963
30000 / 36963
31000 / 36963
32000 / 36963
33000 / 36963
34000 / 36963
35000 / 36963
36000 / 36963
loading kernel1 data...
0 / 6615
1000 / 6615
2000 / 6615
3000 / 6615
4000 / 6615
5000 / 6615
6000 / 6615
loading kernel2 data...
0 / 8640
1000 / 8640
2000 / 8640
3000 / 8640
4000 / 8640
5000 / 8640
6000 / 8640
7000 / 8640
8000 / 8640


In [3]:
    # start IP
    
    # write input, output, kernel1 weight, kernel2 weight address
    ip_r2plus1d.write(0x10, inputBuffer.device_address)
    ip_r2plus1d.write(0x1C, outputBuffer.device_address)
    ip_r2plus1d.write(0x28, kernel1Buffer.device_address)
    ip_r2plus1d.write(0x34, kernel2Buffer.device_address)
    
    # ap_start
    print("IP start")
    ip_r2plus1d.write(0x00, 0x01)
    # wait for the computation to finish
    while (ip_r2plus1d.read(0x00) & 0x4) == 0x0:
        continue
    print("IP done")

IP start
IP done


In [4]:
    OUTPUT = open("/home/xilinx/jupyter_notebooks/R2plus1D/ReLU2output.dat", "r+")
    for i in range(200704):
        line = OUTPUT.readline()
#         err = (outputBuffer[i] - line) / line
        if (outputBuffer[i] != line):
            print("[ERROR]: index", i, 
                  ", result:", outputBuffer[i], 
                  ", gold:", line)
    OUTPUT.close()

[ERROR]: index 0 , result: 0.0 , gold: 0.000000

[ERROR]: index 1 , result: 0.0 , gold: 0.040061

[ERROR]: index 2 , result: 0.0 , gold: 0.434945

[ERROR]: index 3 , result: 0.0 , gold: 0.000000

[ERROR]: index 4 , result: 0.0 , gold: 0.000000

[ERROR]: index 5 , result: 0.0 , gold: 0.697016

[ERROR]: index 6 , result: 0.0 , gold: 0.000000

[ERROR]: index 7 , result: 0.0 , gold: 0.000000

[ERROR]: index 8 , result: 0.0 , gold: 0.064156

[ERROR]: index 9 , result: 0.0 , gold: 1.132434

[ERROR]: index 10 , result: 0.0 , gold: 0.960342

[ERROR]: index 11 , result: 0.0 , gold: 0.189346

[ERROR]: index 12 , result: 0.0 , gold: 1.709820

[ERROR]: index 13 , result: 0.0 , gold: 0.000000

[ERROR]: index 14 , result: 0.0 , gold: 0.000000

[ERROR]: index 15 , result: 0.0 , gold: 0.000000

[ERROR]: index 16 , result: 0.0 , gold: 1.417739

[ERROR]: index 17 , result: 0.0 , gold: 0.000000

[ERROR]: index 18 , result: 0.0 , gold: 0.000000

[ERROR]: index 19 , result: 0.0 , gold: 0.000000

[ERROR]: i

[ERROR]: index 228 , result: 0.0 , gold: 0.000000

[ERROR]: index 229 , result: 0.0 , gold: 0.000000

[ERROR]: index 230 , result: 0.0 , gold: 0.000000

[ERROR]: index 231 , result: 0.0 , gold: 0.000000

[ERROR]: index 232 , result: 0.0 , gold: 0.000000

[ERROR]: index 233 , result: 0.0 , gold: 0.000000

[ERROR]: index 234 , result: 0.0 , gold: 0.000000

[ERROR]: index 235 , result: 0.0 , gold: 0.000000

[ERROR]: index 236 , result: 0.0 , gold: 0.037982

[ERROR]: index 237 , result: 0.0 , gold: 0.203569

[ERROR]: index 238 , result: 0.0 , gold: 0.375594

[ERROR]: index 239 , result: 0.0 , gold: 0.333521

[ERROR]: index 240 , result: 0.0 , gold: 0.000000

[ERROR]: index 241 , result: 0.0 , gold: 0.000000

[ERROR]: index 242 , result: 0.0 , gold: 0.000000

[ERROR]: index 243 , result: 0.0 , gold: 0.000000

[ERROR]: index 244 , result: 0.0 , gold: 0.000000

[ERROR]: index 245 , result: 0.0 , gold: 0.000000

[ERROR]: index 246 , result: 0.0 , gold: 0.000000

[ERROR]: index 247 , result: 0.

[ERROR]: index 459 , result: 0.0 , gold: 0.636146

[ERROR]: index 460 , result: 0.0 , gold: 1.747340

[ERROR]: index 461 , result: 0.0 , gold: 1.183410

[ERROR]: index 462 , result: 0.0 , gold: 1.240232

[ERROR]: index 463 , result: 0.0 , gold: 1.714083

[ERROR]: index 464 , result: 0.0 , gold: 2.516248

[ERROR]: index 465 , result: 0.0 , gold: 0.000000

[ERROR]: index 466 , result: 0.0 , gold: 0.226379

[ERROR]: index 467 , result: 0.0 , gold: 0.000000

[ERROR]: index 468 , result: 0.0 , gold: 0.000000

[ERROR]: index 469 , result: 0.0 , gold: 0.915892

[ERROR]: index 470 , result: 0.0 , gold: 0.000000

[ERROR]: index 471 , result: 0.0 , gold: 0.000000

[ERROR]: index 472 , result: 0.0 , gold: 0.000000

[ERROR]: index 473 , result: 0.0 , gold: 0.416516

[ERROR]: index 474 , result: 0.0 , gold: 0.000000

[ERROR]: index 475 , result: 0.0 , gold: 0.000000

[ERROR]: index 476 , result: 0.0 , gold: 0.000000

[ERROR]: index 477 , result: 0.0 , gold: 0.003871

[ERROR]: index 478 , result: 0.

[ERROR]: index 689 , result: 0.0 , gold: 0.665493

[ERROR]: index 690 , result: 0.0 , gold: 0.000000

[ERROR]: index 691 , result: 0.0 , gold: 1.325014

[ERROR]: index 692 , result: 0.0 , gold: 1.910650

[ERROR]: index 693 , result: 0.0 , gold: 0.365558

[ERROR]: index 694 , result: 0.0 , gold: 0.000000

[ERROR]: index 695 , result: 0.0 , gold: 2.541320

[ERROR]: index 696 , result: 0.0 , gold: 0.000000

[ERROR]: index 697 , result: 0.0 , gold: 0.147157

[ERROR]: index 698 , result: 0.0 , gold: 1.196939

[ERROR]: index 699 , result: 0.0 , gold: 0.624334

[ERROR]: index 700 , result: 0.0 , gold: 0.000000

[ERROR]: index 701 , result: 0.0 , gold: 0.000000

[ERROR]: index 702 , result: 0.0 , gold: 0.000000

[ERROR]: index 703 , result: 0.0 , gold: 0.000000

[ERROR]: index 704 , result: 0.0 , gold: 0.038845

[ERROR]: index 705 , result: 0.0 , gold: 1.393425

[ERROR]: index 706 , result: 0.0 , gold: 0.000000

[ERROR]: index 707 , result: 0.0 , gold: 0.000000

[ERROR]: index 708 , result: 0.

[ERROR]: index 916 , result: 0.0 , gold: 0.342768

[ERROR]: index 917 , result: 0.0 , gold: 0.000000

[ERROR]: index 918 , result: 0.0 , gold: 0.000000

[ERROR]: index 919 , result: 0.0 , gold: 0.000000

[ERROR]: index 920 , result: 0.0 , gold: 0.000000

[ERROR]: index 921 , result: 0.0 , gold: 0.000000

[ERROR]: index 922 , result: 0.0 , gold: 0.000000

[ERROR]: index 923 , result: 0.0 , gold: 0.000000

[ERROR]: index 924 , result: 0.0 , gold: 0.000000

[ERROR]: index 925 , result: 0.0 , gold: 0.092840

[ERROR]: index 926 , result: 0.0 , gold: 0.925360

[ERROR]: index 927 , result: 0.0 , gold: 0.000000

[ERROR]: index 928 , result: 0.0 , gold: 0.000000

[ERROR]: index 929 , result: 0.0 , gold: 0.000000

[ERROR]: index 930 , result: 0.0 , gold: 0.000000

[ERROR]: index 931 , result: 0.0 , gold: 0.000000

[ERROR]: index 932 , result: 0.0 , gold: 0.000000

[ERROR]: index 933 , result: 0.0 , gold: 0.365592

[ERROR]: index 934 , result: 0.0 , gold: 0.000000

[ERROR]: index 935 , result: 0.

[ERROR]: index 1147 , result: 0.0 , gold: 0.000000

[ERROR]: index 1148 , result: 0.0 , gold: 0.144003

[ERROR]: index 1149 , result: 0.0 , gold: 0.000000

[ERROR]: index 1150 , result: 0.0 , gold: 0.807124

[ERROR]: index 1151 , result: 0.0 , gold: 0.000000

[ERROR]: index 1152 , result: 0.0 , gold: 0.279348

[ERROR]: index 1153 , result: 0.0 , gold: 0.694768

[ERROR]: index 1154 , result: 0.0 , gold: 0.000000

[ERROR]: index 1155 , result: 0.0 , gold: 0.000000

[ERROR]: index 1156 , result: 0.0 , gold: 0.000000

[ERROR]: index 1157 , result: 0.0 , gold: 0.000000

[ERROR]: index 1158 , result: 0.0 , gold: 0.210124

[ERROR]: index 1159 , result: 0.0 , gold: 0.793425

[ERROR]: index 1160 , result: 0.0 , gold: 1.402277

[ERROR]: index 1161 , result: 0.0 , gold: 0.240183

[ERROR]: index 1162 , result: 0.0 , gold: 0.785712

[ERROR]: index 1163 , result: 0.0 , gold: 0.000000

[ERROR]: index 1164 , result: 0.0 , gold: 0.000000

[ERROR]: index 1165 , result: 0.0 , gold: 0.000000

[ERROR]: ind

KeyboardInterrupt: 